In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import time

## 1. Load Data

In [2]:
data=pd.read_csv("NY property data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070994 entries, 0 to 1070993
Data columns (total 32 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   RECORD    1070994 non-null  int64  
 1   BBLE      1070994 non-null  object 
 2   B         1070994 non-null  int64  
 3   BLOCK     1070994 non-null  int64  
 4   LOT       1070994 non-null  int64  
 5   EASEMENT  4636 non-null     object 
 6   OWNER     1039249 non-null  object 
 7   BLDGCL    1070994 non-null  object 
 8   TAXCLASS  1070994 non-null  object 
 9   LTFRONT   1070994 non-null  int64  
 10  LTDEPTH   1070994 non-null  int64  
 11  EXT       354305 non-null   object 
 12  STORIES   1014730 non-null  float64
 13  FULLVAL   1070994 non-null  float64
 14  AVLAND    1070994 non-null  float64
 15  AVTOT     1070994 non-null  float64
 16  EXLAND    1070994 non-null  float64
 17  EXTOT     1070994 non-null  float64
 18  EXCD1     638488 non-null   float64
 19  STADDR    1070318 non

## 2. Data Cleaning

### 2.1 Remove properties owned by city, state, or federal governments

In [3]:
# Remove properties owned by city, state, or federal governments
remove_list=['PARKCHESTER PRESERVAT',
            'PARKS AND RECREATION',
            'DCAS',
            'HOUSING PRESERVATION',
            'CITY OF NEW YORK',
            'DEPT OF ENVIRONMENTAL',
            'BOARD OF EDUCATION',
            'NEW YORK CITY HOUSING',
            'CNY/NYCTA',
            'NYC HOUSING PARTNERSH',
            'DEPARTMENT OF BUSINES',
            'DEPT OF TRANSPORTATIO',
            'MTA/LIRR',
            'PARCKHESTER PRESERVAT',
            'MH RESIDENTIAL 1, LLC',
            'LINCOLN PLAZA ASSOCIA',
            'UNITED STATES OF AMER',
            'U S GOVERNMENT OWNRD',
            'THE CITY OF NEW YORK',
            'NYS URBAN DEVELOPMENT',
            'NYS DEPT OF ENVIRONME',
            'CULTURAL AFFAIRS',
            'DEPT OF GENERAL SERVI',
            "DEPT RE-CITY OF NY"]
property_data=data[~data['OWNER'].isin(remove_list)].copy().reset_index()

### 2.2 Filling N/A

#### 2.2.1 ZIP

In [4]:
missing_zips=property_data.index[property_data["ZIP"].isnull()].tolist()
len(missing_zips)

21772

In [5]:
# fill with surrounding zips if the zips before and after the row are the same
for i in missing_zips:
    if(property_data.loc[i+1,'ZIP']==property_data.loc[i-1,'ZIP']):
        property_data.loc[i,'ZIP']=property_data.loc[i-1,'ZIP']
missing_zips=property_data.index[pd.isnull(property_data["ZIP"])].tolist()
len(missing_zips)

10245

In [6]:
# fill with preceding ZIP
for i in missing_zips:
    property_data.loc[i,'ZIP']=property_data.loc[i-1,'ZIP']
missing_zips=property_data.index[pd.isnull(property_data["ZIP"])].tolist()
len(missing_zips)

0

#### 2.2.2 FULLVAL, AVLAND, AVTOT

In [7]:
# mean of FULLVAL by TAXCLASS
property_data["FULLVAL"]=property_data["FULLVAL"].replace('NaN',0)
temp=property_data[property_data["FULLVAL"]!=0]
mean_fullval=temp.groupby("TAXCLASS")["FULLVAL"].mean()
print(mean_fullval)

TAXCLASS
1     5.700059e+05
1A    3.352942e+05
1B    3.738399e+05
1C    7.615359e+05
1D    2.233614e+07
2     8.005832e+05
2A    8.639066e+05
2B    1.254525e+06
2C    7.723493e+05
3     8.510005e+04
4     2.772747e+06
Name: FULLVAL, dtype: float64


In [8]:
# mean of AVLAND by TAXCLASS
property_data["AVLAND"]=property_data["AVLAND"].replace('NaN',0)
temp_avland=property_data[property_data["AVLAND"]!=0]
mean_avland=temp.groupby("TAXCLASS")["AVLAND"].mean()

In [9]:
# mean of AVTOT by TAXCLASS
property_data["AVTOT"]=property_data["AVTOT"].replace('NaN',0)
temp_avtot=property_data[property_data["AVTOT"]!=0]
mean_avtot=temp.groupby("TAXCLASS")["AVTOT"].mean()

In [10]:
# sustitute 0 with averages by TAXCLASS
%time
for index in mean_fullval.index:
    property_data.loc[(property_data["FULLVAL"]==0)&(property_data["TAXCLASS"]==index),"FULLVAL"]=mean_fullval[index]
    property_data.loc[(property_data["AVLAND"]==0)&(property_data["TAXCLASS"]==index),"AVLAND"]=mean_avland[index]
    property_data.loc[(property_data["AVTOT"]==0)&(property_data["TAXCLASS"]==index),"AVTOT"]=mean_avtot[index]
    

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs


#### 2.2.3 STOREIS

In [11]:
temp=property_data[property_data["STORIES"].isnull()]
print(f"Num of Missing values:{len(temp)}")
print("Num of 0 stories: {}".format((property_data["STORIES"]==0).sum()))
temp["TAXCLASS"].value_counts()

Num of Missing values:43968
Num of 0 stories: 0


1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [12]:
mean_stories=property_data.groupby("TAXCLASS")["STORIES"].mean()
mean_stories

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64

In [13]:
property_data["STORIES"]=property_data["STORIES"].fillna(0)
for index in mean_stories.index:
    property_data.loc[(property_data["STORIES"]==0)&(property_data["TAXCLASS"]==index),'STORIES']=mean_stories[index]

#### 2.2.4 LTFRONT, LTDEPTH, BLDFRONT, BLDDEPTH

In [14]:
# as these 4 values do not have NAs, we need to replace 0s and 1s
# calculate groupwise average. First replace 0's and 1's by NAs so they are not counted in calculating mean.
property_data.loc[property_data["LTFRONT"]==0,"LTFRONT"]=np.nan
property_data.loc[property_data["LTFRONT"]==1,"LTFRONT"]=np.nan
property_data.loc[property_data["LTDEPTH"]==0,"LTDEPTH"]=np.nan
property_data.loc[property_data["LTDEPTH"]==1,"LTDEPTH"]=np.nan
property_data.loc[property_data["BLDFRONT"]==0,"BLDFRONT"]=np.nan
property_data.loc[property_data["BLDFRONT"]==1,"BLDFRONT"]=np.nan
property_data.loc[property_data["BLDDEPTH"]==0,"BLDDEPTH"]=np.nan
property_data.loc[property_data["BLDDEPTH"]==1,"BLDDEPTH"]=np.nan

In [15]:
# calculate the mean 
mean_LTFRONT=property_data.groupby("TAXCLASS")["LTFRONT"].mean()
mean_LTDEPTH=property_data.groupby("TAXCLASS")["LTDEPTH"].mean()
mean_BLDFRONT=property_data.groupby("TAXCLASS")["BLDFRONT"].mean()
mean_BLDDEPTH=property_data.groupby("TAXCLASS")["BLDDEPTH"].mean()

In [16]:
# impute values
for index in mean_LTFRONT.index:
    property_data.loc[(property_data["LTFRONT"].isnull())&(property_data["TAXCLASS"]==index),"LTFRONT"]=mean_LTFRONT[index]
    property_data.loc[(property_data["LTDEPTH"].isnull())&(property_data["TAXCLASS"]==index),"LTDEPTH"]=mean_LTDEPTH[index]
    property_data.loc[(property_data["BLDFRONT"].isnull())&(property_data["TAXCLASS"]==index),"BLDFRONT"]=mean_BLDFRONT[index]
    property_data.loc[(property_data["BLDDEPTH"].isnull())&(property_data["TAXCLASS"]==index),"BLDDEPTH"]=mean_BLDDEPTH[index]

## 3. Create Variables

In [17]:
property_data["ZIP"]=property_data["ZIP"].astype(str)
property_data["zip3"]=property_data["ZIP"].str[:3]

In [18]:
property_data["ltsize"]=property_data["LTFRONT"]*property_data["LTDEPTH"]
property_data["bldsize"]=property_data["BLDFRONT"]*property_data["BLDDEPTH"]
property_data["bldvol"]=property_data["bldsize"]*property_data["STORIES"]

In [19]:
property_data['r1']=property_data["FULLVAL"]/property_data["ltsize"]
property_data['r2']=property_data["FULLVAL"]/property_data["bldsize"]
property_data['r3']=property_data["FULLVAL"]/property_data["bldvol"]
property_data['r4']=property_data["AVLAND"]/property_data["ltsize"]
property_data['r5']=property_data["AVLAND"]/property_data["bldsize"]
property_data['r6']=property_data["AVLAND"]/property_data["bldvol"]
property_data['r7']=property_data["AVTOT"]/property_data["ltsize"]
property_data['r8']=property_data["AVTOT"]/property_data["bldsize"]
property_data['r9']=property_data["AVTOT"]/property_data["bldvol"]

In [20]:
ninevars=['r1','r2','r3','r4','r5','r6','r7','r8','r9']
zip5_mean=property_data.groupby('ZIP')[ninevars].mean()
zip3_mean=property_data.groupby('zip3')[ninevars].mean()
taxclass_mean=property_data.groupby('TAXCLASS')[ninevars].mean()
borough_mean=property_data.groupby("B")[ninevars].mean()

property_data=property_data.join(zip5_mean,on='ZIP',rsuffix='_zip5')
property_data=property_data.join(zip3_mean,on='zip3',rsuffix='_zip3')
property_data=property_data.join(taxclass_mean,on='TAXCLASS',rsuffix='_taxclass')
property_data=property_data.join(borough_mean,on='B',rsuffix='_boro')

In [21]:
rsuffix=['_zip5','_zip3','_taxclass','_boro']
for var in ninevars:
    for r in rsuffix:
        property_data[str(var)+r]=property_data[var]/property_data[str(var)+r]

In [22]:
property_data.loc[:,property_data.columns[37:]].describe().T.to_excel("test.xlsx")